# Import packages and set parameters

In [ ]:
import numpy as np
from numba import jit

import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors

from scipy.interpolate import RectBivariateSpline

import os
import pandas as pd

def set_pandas_display_options() -> None:
    """Set pandas display options."""
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = None
    display.max_rows = None
    display.max_colwidth = None
    display.width = None
    display.precision = None  # set as needed
    display.float_format = '{:,.8f}'.format

# Define methods

## DOS interpolation

In [ ]:
@jit(nopython=True)
def fermi(E,mu,kbT):
    return 1.0 / ( np.exp( (E-mu) / kbT) + 1)

@jit(nopython=True)
def dfermi(E,mu,kbT):
    return 1.0/(2*kbT + 2*kbT*np.cosh((E - mu)/kbT))

@jit(nopython=True)
def dos(mu, kbT, spec):
       
    dens = 0.0
    
    for level in spec:
        dens += dfermi(level,mu,kbT)
        
    return dens / ( len(spec) / 2.0)

@jit(nopython=True)
def ids(mu, spec):
       
    dens = 0.0 
    
    for level in spec:
        if level < mu:
            dens += 1
        
    return dens / ( len(spec) / 2.0)

# Import Data

## Read files for parameters

In [ ]:
files = [x[0] for x in os.walk('./data/')][1::]

df = pd.DataFrame()
for file in files:
    data = pd.read_json(file+'/params.json',orient='index').transpose()
    data["key"] = file[7:]
    
    if data.at[0,'texture'] != "skx":
        continue
    
    df = pd.concat([df,data])

In [ ]:
display(df.set_index('key').sort_index(ascending=False))

## Select data

In [ ]:
stamp='1746309131'

## Import file and extract data

In [ ]:
outdir = './data/'+stamp

# Thetas and system sizes
qs = np.load(outdir+"/qs.npy")
ns = np.load(outdir+"/ns.npy")
n_q = len(qs)

# Full theta range
qs = np.concatenate((qs,np.array([1-q for q in qs[::-1]])))
ns = np.concatenate((ns,ns[::-1]))

# Full spectrum
spec = []
for i in range(n_q):
    spec.append( np.load(outdir+"/spec_"+str(i).zfill(4)+".npy") )
for i in range(n_q-1,-1,-1):
    spec.append( np.load(outdir+"/spec_"+str(i).zfill(4)+".npy") )
    
n_q =len(qs)

# Interpolate spectrum

In [ ]:
Emins = np.zeros(n_q)
Emaxs  = np.zeros(n_q)
for i in range(n_q):
    Emins[i] = np.amin(spec[i])
    Emaxs[i] = np.amax(spec[i])
    
Emin = np.amin(Emins)
Emax = np.amax(Emaxs)

nmu = 500
kbT = 0.02
mus = np.linspace(Emin,Emax,nmu)

Q   = np.zeros(nmu*n_q)
DQ  = np.zeros(nmu*n_q)
DOS = np.zeros(nmu*n_q)
IDS = np.zeros(nmu*n_q)
MU  = np.zeros(nmu*n_q)

k = 0 
for i in range(nmu):
    for j in range(n_q):
        
        Q[k]   = qs[j]
        if j+1 < n_q:
            DQ[k]  = qs[j+1] - qs[j]
        IDS[k] = ids(mus[i], spec[j]) 
        DOS[k] = dos(mus[i], kbT, spec[j]) 
        MU[k]  = mus[i]
        
        k += 1
        
interpolation_order = 1
fids   = RectBivariateSpline(mus, qs, np.reshape(IDS, (nmu,n_q)), kx=interpolation_order, ky=interpolation_order) 
fdos   = RectBivariateSpline(mus, qs, np.reshape(DOS, (nmu,n_q)), kx=interpolation_order, ky=interpolation_order) 
fqdens = RectBivariateSpline(mus, qs, np.reshape(DQ,  (nmu,n_q)), kx=interpolation_order, ky=interpolation_order)

n_qi  = 500
n_mui = 500

qi = np.linspace(np.amin(qs),np.amax(qs),n_qi)

mui = np.linspace(Emin,Emax,n_mui)

Qi, MUi = np.meshgrid(qi,mui,indexing='ij')
DOSi    = np.zeros( (n_qi, n_mui))
IDSi    = np.zeros( (n_qi, n_mui))
QDENSi  = np.zeros( (n_qi, n_mui))

for i in range(n_qi):
    for j in range(n_mui):    
        DOSi[i,j]   = np.abs(fdos( MUi[i,j], Qi[i,j]))
        IDSi[i,j]   = np.abs(fids( MUi[i,j], Qi[i,j]))
        QDENSi[i,j] = np.abs(fqdens( MUi[i,j], Qi[i,j]))
        
mindens = np.amax(QDENSi)
maxdens = np.amax(QDENSi)

# Plots

In [ ]:
set_pandas_display_options()
plt.rcParams['figure.figsize'] = [8, 6]
plt.rcParams['savefig.facecolor'] = "white"
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams['axes.linewidth'] = 1.2
mpl.rcParams['text.usetex'] = True
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['font.family'] = 'cmu serif'
tfs    = 30 #title font size
lfs    = 20 #label font size
fs     = 18 #font size
cbarfs = 20 #colorbar font size

## DOS

In [ ]:
x_ticks = np.arange(0, 1.1, 0.1)
y_ticks = np.arange(-10, 10, 2.5)

plt.pcolormesh(Qi,MUi,DOSi, cmap='Blues',norm=colors.LogNorm(vmin=0.009, vmax=3.0))

ax = plt.gca()
ax.set_xlabel(r"$\vartheta$",fontsize=lfs)
ax.set_ylabel(r"$E_F \; / \; \lambda_\mathrm{hop}$",fontsize=lfs,labelpad=-5)
ax.set_xticks(x_ticks)
ax.set_yticks(y_ticks)
ax.tick_params(axis='both', which='major', labelsize=fs)
#ax.set_xlim(0,1)
ax.set_ylim(-11,8)

cbar = plt.colorbar(pad=0.02)
cbar.set_label(label=r'DOS (a.u.)',size=cbarfs)
cbar.ax.tick_params(labelsize=fs)

# attaching gap labels
gaps = np.array( [ [0.175, -9.94],[0.175, -9.42], [0.175, -1.21], [0.175, 0.27], [0.175, 0.92], [0.175, 1.48] ] )
gaps_col = cm.plasma(np.linspace(0,1,len(gaps)))
plt.scatter(gaps[:,0],gaps[:,1],c=gaps_col,s=50)

plt.tight_layout()
plt.savefig("./plots/skx_DOS", dpi=300, bbox_inches = 'tight')

## IDS

In [ ]:
gaps_q = gaps[:,0]
gaps_mu = gaps[:,1]
gaps_ids = np.array( [ fids( gaps_mu[i], gaps_q[i]) for i in range(len(gaps)) ] )

In [ ]:
x_ticks = np.arange(0, 1.1, 0.1)
y_ticks = np.arange(0, 2.5, 0.5)

plt.pcolormesh(Qi,IDSi,MUi,cmap='RdBu')

ax = plt.gca()
ax.set_xlabel(r"$\vartheta$",fontsize=lfs)
ax.set_ylabel(r"$IDS$",fontsize=lfs)
ax.set_xticks(x_ticks)
ax.set_yticks(y_ticks)
ax.tick_params(axis='both', which='major', labelsize=fs)
#ax.set_xlim(0,1)
ax.set_ylim(0,2)

cbar = plt.colorbar(pad=0.02)
cbar.set_label(label=r'$E_F \; / \; \lambda_\mathrm{hop}$', size=lfs)
cbar.ax.tick_params(labelsize=fs)

tups = [ [0,0,1], [0,0,2], [1,0,0], [1,0,1], [1,0,2], [1,0,3] ]

x = np.linspace(qs[0],qs[n_q//2],100)

for tup in tups:
    m, n , k = tup
    y = ( m + n*x + k*x**2 ) 
    plt.plot(x,y, color='black', linewidth=.5, linestyle='--', dashes=(10, 30))

plt.scatter(gaps_q, gaps_ids,c=gaps_col, s=30, zorder=10)

ax.annotate('0,0,1', xy=(0.45, 0.12), rotation=0, fontsize=fs)
ax.annotate('0,0,2', xy=(0.45, 0.32), rotation=0, fontsize=fs)
ax.annotate('1,0,0', xy=(0.45, 0.92), rotation=0, fontsize=fs)
ax.annotate('1,0,1', xy=(0.45, 1.12), rotation=0, fontsize=fs)
ax.annotate('1,0,2', xy=(0.45, 1.32), rotation=0, fontsize=fs)
ax.annotate('1,0,3', xy=(0.45, 1.52), rotation=0, fontsize=fs)

plt.tight_layout()
plt.savefig("./plots/skx_IDS", dpi=300, bbox_inches = 'tight')